# Sentify: Your Emotion-Driven Spotify Player

Discover Sentify, an innovative AI-powered Spotify player that tunes into your emotions. With advanced emotion recognition technology, Sentify instantly analyzes your mood and generates personalized Spotify playlists that perfectly match your emotions. Elevate your music experience and immerse yourself in tracks that resonate with your feelings, whether you're feeling upbeat, relaxed, or nostalgic. Say hello to an intuitive and personalized music journey with Sentisonics.

**Key Features:**
- Emotion-Driven Playlists
- Real-Time Mood Analysis
- Seamless Spotify Integration
- Intuitive User Interface
- Personalized Music Recommendations

#### Experience the future of music curation with Sentify and let your emotions lead the way!

### Importing libraries and modules 

In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import json
import spotipy
import webbrowser
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.16.2


- I have imported json as i need to save the model after training in json format. Moreover i used spotipy which is used for spoify api and integration which can be generated using spotify deveoper site . 
- You can check it out and have fun with your spotify client id and client secret. 
- I imported webbrowser as the model not only just fetch the playlist and display , but also it playlost URI and redirect the user to the particular spotify playlist. If the spotify is predownloaded in system , then the playlist will open in spotify app else it will open in your rediected browser. 
- I used tensorflow with keras in order to train the model. I used adam optimizer which suits best for this project.  

### Importing every folder in train section and printing number of training images

In [2]:
for expression in os.listdir("C:/Users/hardik gupta/Desktop/CAREER/sentisonics/facial_expression/train/"):
    print(str(len(os.listdir("C:/Users/hardik gupta/Desktop/CAREER/sentisonics/facial_expression/train/" + expression))) + " " + expression + " images")

3993 angry images
436 disgust images
4103 fear images
7164 happy images
4982 neutral images
4938 sad images
3205 surprise images


### Integrating the data into categorical form

In [ ]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("C:/Users/hardik gupta/Desktop/CAREER/sentisonics/facial_expression/train/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_validation.flow_from_directory("C:/Users/hardik gupta/Desktop/CAREER/sentisonics/facial_expression/train/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

## Initializing the CNN model

In [ ]:
model = Sequential()
# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# Flattening
model.add(Flatten())
# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))
opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

- In the CNN (convolutional neural network) we took 4 convolutional layer with relu activation function, max pooling of size (2,2). 
- As we took batch size , we have taken batch normalization in every convolutional layer. 
- After that , we used flattening layer which will flatten our model. 
- Next, we took 2 fully connected layers 1st with 256 neurons and the 2nd with 512 neurons with activation function of relu and dropout of 0.25.
- At last we added a softmax function to reduce the noise.
- In this model we took adam optimizer with learning rate of 0.0005.
- Finally we compiled the model with these categories and printed the summary of the model. 

In [ ]:
%%time
epochs = 15
steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')
checkpoint = ModelCheckpoint("model_weights.weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
callbacks = [PlotLossesCallback(), checkpoint, reduce_lr]
history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps,
    callbacks=callbacks
)

Here we trained the model with above mentioned characteristics and saved the model as model.json and its updated weights as model_weights.h5. Here we can see that our model is providing with 66% accuracy. Not too good but not too bad. I have taken 15 epochs , hence in order to improve the model accuracy, It can be increased.

In [ ]:
model_json = model.to_json()
model.save_weights('model_weights.weights.h5')
with open("model.json", "w") as json_file:
    json_file.write(model_json)

- Now our model is trained and saved along with its updated weights.
- Hence , we need to set up the camera and test for the gestures and also along side we will setup our spotify and integrate it with our model.

### Created Facial Expression model class

In [7]:
from tensorflow.keras.models import model_from_json
class FacialExpressionModel(object):
    EMOTIONS_LIST = ["Angry", "Disgust",
                    "Fear", "Happy",
                    "Neutral", "Sad",
                    "Surprise"]
    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)
        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        self.loaded_model.make_predict_function()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)
        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]

This class will take the object as an argument. 
- Main function of this class is to nuild the emotion list , Extract the model from json format , read the model.
- The second function predicts the emotion on the basis of the prediction returned by the model .

In [8]:
import cv2
global label
facec = cv2.CascadeClassifier('C:/Users/hardik gupta/Desktop/CAREER/sentisonics/facial_expression/haarcascade_frontalface_default.xml')
model = FacialExpressionModel("C:/Users/hardik gupta/Desktop/CAREER/sentisonics/model.json", "C:/Users/hardik gupta/Desktop/CAREER/sentisonics/model_weights.weights.h5")
font = cv2.FONT_HERSHEY_SIMPLEX
class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)
    def __del__(self):
        self.video.release()
    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)
        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]
            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])
            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)
            if cv2.waitKey(1) & 0xFF == ord('s'):
                label = pred
                play(label)
                print(label)
        return fr

This class basically handles the Camera and draws the square over the face and return print the predicted expression over the box.
- This calls the other function play which will take the predicted emotion in real time and redirect to the particular playlist.

## Spotify setup and Integration

In [9]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
username = 'Chichi'
clientID = 'cb5910d004c94f5092a4f8c901d15148'
clientSecret = 'e0132a3de43e44c380c3233c6e5ee42f'
redirect_uri = 'http://google.com/callback/'

I have took the redirected uri as 'http://google.com/callback/' as it uses the google for redirect purposes.
- Make sure you write the same redirect uri as you write in spotify developer project.

In [10]:
import json
oauth_object = spotipy.SpotifyOAuth(clientID, clientSecret, redirect_uri)
token_dict = oauth_object.get_access_token()
token = token_dict['access_token']
spotifyObject = spotipy.Spotify(auth=token)
user_name = spotifyObject.current_user()
  
# To print the response in readable format.
print(json.dumps(user_name, sort_keys=True, indent=4))

C:\Users\hardik gupta\AppData\Local\Temp\ipykernel_18748\4044356168.py:3: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_dict = oauth_object.get_access_token()


{
    "display_name": "Chichi",
    "external_urls": {
        "spotify": "https://open.spotify.com/user/316sky2fqi4kii3gjuzsrptlftwe"
    },
    "followers": {
        "href": null,
        "total": 0
    },
    "href": "https://api.spotify.com/v1/users/316sky2fqi4kii3gjuzsrptlftwe",
    "id": "316sky2fqi4kii3gjuzsrptlftwe",
    "images": [],
    "type": "user",
    "uri": "spotify:user:316sky2fqi4kii3gjuzsrptlftwe"
}


- A general setup for spotipy. It takes the client id, client secret, redirect uri in oath object.
- It gets the access token. Remember the spotify will ask for the access permission.
- After all this, the code will redirect you to new link to get access. If already given access, then it will generate the json file of your account. If not then it will ask for the link to which you were redirected now.

In [11]:
def play(mood):
    if mood == 'Angry':
        song = 'https://open.spotify.com/playlist/37i9dQZF1EIgNZCaOGb0Mi?si=9cd00a9874124523'
        
    elif mood == 'Disgust':
        song = 'https://open.spotify.com/album/580A6nn3HTfoqJGmPgCRum?si=usttXe_oT2W4XGu6VvTZEA'
        
    elif mood == 'Fear':
        song = 'https://open.spotify.com/album/2BWtI3RIIf5V9uXr7B3vQd?si=osDGyD1MRX2VSBu7Y8VOpg'
        
    elif mood == 'Happy':
        song = 'https://open.spotify.com/playlist/0jrlHA5UmxRxJjoykf7qRY?si=d97f5386d2644071'
        
    elif mood == 'Neutral':
        song = 'https://open.spotify.com/album/5V2zxxg2ZQe8cpLlRxsCFr?si=qHHQx5FES9iKIKvKuZudqg'
        
    elif mood == 'Sad':
        song = 'https://open.spotify.com/album/71sb3x5BwKk87d7ZBOoSJY?si=0UN2cEOPR8WwSehAYlaRSA'
        
    elif mood == 'Surprise':
        song = 'https://open.spotify.com/playlist/3oiDdYrRCuqu5e4CgwqVkE?si=b008af6a30d942ef'
    
    webbrowser.open(song)

This function is basically the heart of the whole model, as this function takes the mood checks which type of mood it is and then according to mood , it stores the particular playlist / album uri and opens it to your redirected webbrowser.

In [12]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

gen function is the head of the model . It calls all the classes and functions which were discussed earlier.

### Key controls :
#### 's' to capture the emotion and search for the playlist.
#### 'q' to quit the camera and exit the model.

In [ ]:
gen(VideoCamera())

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━

The model recognizes every single frame of your emotion.